In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
from tqdm.notebook import tqdm
import scipy.stats as stats

import polyRBM
import analysisUtility as anaU

### helper functions

In [ ]:
def not_in_set(dataset, recon):
    not_in_set = 0
    unD = np.unique(dataset, axis=0)
    unR, unR_count = np.unique(recon.astype(int), return_counts=True, axis=0)
    for ind, u in tqdm(enumerate(unR)):
        boo = (u==unD).all(axis=1)
        ind = np.where(boo==True)[0]
        if ind.size > 0:
            not_in_set += unR_count[ind[0]]
    return not_in_set/len(recon)

def sample_loops(savepath, run, N, ete=False):
    rbm = anaU.loader(savepath, run, rbm=True)
    all_samples = []
    
    for param in rbm.parameters():
        param.requires_grad = False
    print(f'start sampling from {run} RBM')
    for i in tqdm(range(10)):
        initialize = torch.from_numpy(np.random.binomial(1, 0.5, (1000, (N-2)*2))).float()
        rbm.k = 800
        samples = rbm(initialize)[2].detach().numpy()
        all_samples.append(samples)
    
    samples = np.concatenate(all_samples)
    if ete==True:
        samplesTF = polyRBM.bitsToBonds2DYu(samples)
        ete_vec = np.sqrt(np.sum(np.sum(samplesTF, axis=1)**2, axis=1))
        endtoend = np.mean(ete_vec)
    refloops = anaU.loop_count(samples)
    if ete==True:
        return refloops, endtoend
    return refloops

def bitwise_dist(data, rbm, kmax):
    bitwise_dist = np.zeros(kmax)
    if len(data)>10000:
        data = data[:10000]
    for k in tqdm(range(kmax)):
        rbm.k = k+1
        if k<100:
            repro = rbm(data)[2].detach().numpy()
            dist = np.sum(abs(repro-data.detach().numpy()), axis=1)
            bitwise_dist[k] = np.mean(dist)
        elif k<300 and k%10==0:
            repro = rbm(data)[2].detach().numpy()
            dist = np.sum(abs(repro-data.detach().numpy()), axis=1)
            bitwise_dist[k] = np.mean(dist)
        elif k>=300 and k%100==0:
            repro = rbm(data)[2].detach().numpy()
            dist = np.sum(abs(repro-data.detach().numpy()), axis=1)
            bitwise_dist[k] = np.mean(dist)
        else:
            bitwise_dist[k] = None
    return bitwise_dist

def interpolate_gaps(values, limit=None):
    """
    Fill gaps using linear interpolation, optionally only fill gaps up to a
    size of `limit`.
    """
    values = np.asarray(values)
    i = np.arange(values.size)
    valid = np.isfinite(values)
    filled = np.interp(i, i[valid], values[valid])

    if limit is not None:
        invalid = ~valid
        for n in range(1, limit+1):
            invalid[:-n] &= invalid[n:]
        filled[invalid] = np.nan
    return filled

def bitwise_dist_twoSets(data):
    length = int(len(data)/2)
    dat1 = data[:length]
    dat2 = data[length:]
    if len(dat1)>len(dat2):
        dat1 = dat1[:len(dat2)]
    elif len(dat2)>len(dat1):
        dat2 = dat2[:len(dat1)]
    print(np.sum(abs(dat1-dat2), axis=1))
    dist = np.mean(np.sum(abs(dat1-dat2), axis=1))
    
    return dist/len(dat1[0, :])

def interpolate_gaps(values, limit=None):
    """
    Fill gaps using linear interpolation, optionally only fill gaps up to a
    size of `limit`.
    """
    values = np.asarray(values)
    i = np.arange(values.size)
    valid = np.isfinite(values)
    filled = np.interp(i, i[valid], values[valid])

    if limit is not None:
        invalid = ~valid
        for n in range(1, limit+1):
            invalid[:-n] &= invalid[n:]
        filled[invalid] = np.nan

    return filled

## evaluation of ring conformations

In [ ]:
savepath_8 = ###directory of N=8 training results###
savepath_32 = ###directory of N=32 training results###

run8 = ###unique identifier of training run for N=8###
run32 = ###unique identifier of training run for N=32###

fname_8 = ###file path of N=8 simulation data###
fname_32 = ###file path of N=32 simulation data###

### loading of reconstructions & calculation of observables

In [ ]:
Ns = [8, 32]
list8 = []
list32 = []
lists = [list8, list32]
runs = [run8, run32]
savepaths = [savepath_8, savepath_32]
refloops = []
fnames = [fname_8, fname_32]

for enum, N in enumerate(Ns):
    # create sample data from trained RBM
    rbm = anaU.loader(savepaths[enum], runs[enum], rbm=True)
    all_samples = []
    for param in rbm.parameters():
        param.requires_grad = False
    
    print(f'start sampling from {runs[enum]} RBM')
    for i in tqdm(range(10)):
        initialize = torch.from_numpy(np.random.binomial(1, 0.5, (1000, (N+1-2)*2))).float()
        rbm.k = 800
        samples = rbm(initialize)[2].detach().numpy()
        all_samples.append(samples)
    
    samples = np.concatenate(all_samples)
    samplesTF = polyRBM.makeVecs(polyRBM.bitsToBonds2DYu(samples))

    # load raw data
    dataset = polyRBM.CubicChainDataset(fnames[enum], 2)

    ind = np.linspace(0, len(dataset)-1, len(dataset)).astype(int)
    np.random.shuffle(ind)

    dataset.data = dataset.data[ind]
    dataset.bits = dataset.bits[ind]
    dataset.bonds = dataset.bonds[ind]
    
    ringbonds = np.zeros((len(dataset.bonds[:, 0, 0]), len(dataset.bonds[0, :, 0])+1, len(dataset.bonds[0, 0, :])))
    ringbonds[:, :-1, :] = dataset.bonds
    ringbonds[:, -1, :] = -np.cumsum(dataset.bonds, axis=1)[:, -1, :]
    dataset.bonds = ringbonds
    dataset.bits = polyRBM.bondsToBits2DYu(dataset.bonds)
    dataset.data = torch.from_numpy(dataset.bits).float()
    
    total = int(1*len(dataset)) 
    div = int(0.7*total)
    testset = dataset.bits[div:total]
    dataset.data = dataset.data[:div]
    dataset.bonds = dataset.bonds[:div]
    dataset.bits = dataset.bits[:div]
    
    # calculate observables 
    bwise_dist_dat = bitwise_dist_twoSets(dataset.data.detach().numpy())
    bwise_dist = bitwise_dist(dataset.data, rbm, 1)
    overlap = not_in_set(dataset.data.detach().numpy(), samples)
    
    data = polyRBM.makeVecs(dataset.bonds)
    
    refloops.append(anaU.loop_count(dataset.bits)[anaU.loop_count(dataset.bits)!=0])
    
    lists[enum].append(dataset.bits)
    lists[enum].append(data)
    lists[enum].append(samples)
    lists[enum].append(samplesTF) 
    lists[enum].append(bwise_dist_dat)
    lists[enum].append(bwise_dist)
    lists[enum].append(overlap)

### calculation of observables and plot

In [ ]:
# calculate bondvector correlation and Rg^2 moments
plt.rcParams.update({'font.size': 18})
N8N32_exclVol_corr = []
N8N32_exclVol_moments = []
N8N32_exclVol_moments_dev = []

for l in lists:
    N8N32_exclVol_corr.append(anaU.bondvec_correlation(l[1], len(l[1]))) #data
    N8N32_exclVol_corr.append(anaU.bondvec_correlation(l[3], len(l[3]))) #samples
    
    rg2_samples = anaU.rgx_calc(l[3], len(l[3]))
    rg2_data = anaU.rgx_calc(l[1], len(l[1]))

    tp1S = np.mean(rg2_samples[-1])
    tp1S_dev = np.sqrt(np.sum((tp1S - rg2_samples[-1])**2)/(len(rg2_samples[-1])-1))
    tp1D = np.mean(rg2_data[-1])
    tp2S = np.mean(rg2_samples[-1]**2)
    tp2S_dev = np.sqrt(np.sum((tp2S - rg2_samples[-1]**2)**2)/(len(rg2_samples[-1])-1))
    tp2D = np.mean(rg2_data[-1]**2)
    tp3S = np.mean(rg2_samples[-1]**3)
    tp3S_dev = np.sqrt(np.sum((tp2S - rg2_samples[-1]**3)**2)/(len(rg2_samples[-1])-1))
    tp3D = np.mean(rg2_data[-1]**3)

    momentsS = [tp1S, tp2S, tp3S]
    momentsS_dev = [tp1S_dev, tp2S_dev, tp3S_dev]
    momentsD = [tp1D, tp2D, tp3D]
    N8N32_exclVol_moments.append(momentsD)
    N8N32_exclVol_moments.append(momentsS)
    N8N32_exclVol_moments_dev.append(momentsS_dev)
    
# loop plot
loops_8 = anaU.loader(savepaths[0], run8, loops=True)
loops_32 = anaU.loader(savepaths[1], run32, loops=True)

not_zero_8 = np.logical_not((np.array(loops_8)==0).all(axis=0))
not_zero_32 = np.logical_not((np.array(loops_32)==0).all(axis=0))
loop8 = np.array(loops_8)[:, not_zero_8]
loop32 = np.append((np.array(loops_32)[:, not_zero_32])[:, :4], (np.array(loops_32)[:, not_zero_32])[:, -1:], axis=1)
fig, axs = plt.subplots(2)
axs[0].imshow(loop8[:10])
axs[1].imshow(loop32[:10])

fig_loops, ax_loops = plt.subplots(1, 2)
rbm_epochs = len(loop8)
rbm_epochs_big = len(loop32)

for i in range(len(loop8[0, :])):
    loop8[:, i][loop8[:, i]==0] = None
    ax_loops[0].plot(np.linspace(0, rbm_epochs, rbm_epochs), loop8[:, i], label=f'l={np.where(not_zero_8==True)[0][i]}')
    
for i in range(len(loop32[0, :])):
    if i+1==len(loop32[0, :]):
        ax_loops[1].plot(np.linspace(0, rbm_epochs_big, rbm_epochs_big), loop32[:, i], label='l=32')
    else:
        ax_loops[1].plot(np.linspace(0, rbm_epochs_big, rbm_epochs_big), loop32[:, i], label=f'l={np.where(not_zero_32==True)[0][i]}')
           
ax_loops[0].set_title('N=8')
ax_loops[1].set_title('N=32')
ax_loops[0].set_xlim(-2, rbm_epochs)
ax_loops[1].set_xlim(-2, rbm_epochs_big)
ax_loops[0].set_xlabel('epochs')
ax_loops[0].set_ylabel('loops per chain')
ax_loops[1].set_xlabel('epochs')
ax_loops[1].set_ylabel('loops per chain')
ax_loops[0].legend(loc='upper right', bbox_to_anchor=(1,0.8))
ax_loops[1].legend(loc='upper right', bbox_to_anchor=(1,0.9))
fig_loops.set_size_inches(10, 5)
fig_loops.tight_layout()

# bonvec plot
fig_corr, ax_corr = plt.subplots(1, 2)
ax_corr[0].plot(np.linspace(0, len(N8N32_exclVol_corr[0])-1, len(N8N32_exclVol_corr[0])), N8N32_exclVol_corr[0])
ax_corr[0].scatter(np.linspace(0, len(N8N32_exclVol_corr[0])-1, len(N8N32_exclVol_corr[0])), N8N32_exclVol_corr[0],
                  label='data')
ax_corr[0].plot(np.linspace(0, len(N8N32_exclVol_corr[1])-1, len(N8N32_exclVol_corr[1])), N8N32_exclVol_corr[1])
ax_corr[0].scatter(np.linspace(0, len(N8N32_exclVol_corr[1])-1, len(N8N32_exclVol_corr[1])), N8N32_exclVol_corr[1],
                  label='reconstruction')
ax_corr[0].set_title('N=8')
ax_corr[0].set_xlabel('|i-j|')
ax_corr[0].set_ylabel('$\\left\\langle\\frac{\\vec{b}_i \cdot \\vec{b}_j}{|\\vec{b}_i|\cdot|\\vec{b}_j|}\\right\\rangle$')
ax_corr[0].legend()

ax_corr[1].plot(np.linspace(0, len(N8N32_exclVol_corr[2])-1, len(N8N32_exclVol_corr[2])), N8N32_exclVol_corr[2])
ax_corr[1].scatter(np.linspace(0, len(N8N32_exclVol_corr[2])-1, len(N8N32_exclVol_corr[2])), N8N32_exclVol_corr[2])
ax_corr[1].plot(np.linspace(0, len(N8N32_exclVol_corr[-1])-1, len(N8N32_exclVol_corr[-1])), N8N32_exclVol_corr[3])
ax_corr[1].scatter(np.linspace(0, len(N8N32_exclVol_corr[-1])-1, len(N8N32_exclVol_corr[-1])), N8N32_exclVol_corr[3])
ax_corr[1].set_title('N=32')
ax_corr[1].set_xlabel('|i-j|')
ax_corr[0].set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
ax_corr[1].set_xticks([0, 5, 10, 15, 20, 25, 30])

fig_corr.set_size_inches(10, 5)
fig_corr.tight_layout()

# Rg^2 moments plot
xs = [2, 4, 6]
colors = ['red', 'orange', 'green']
fig_rg, ax_rg = plt.subplots(1, 2)
ax_rg[0].scatter(xs, N8N32_exclVol_moments[1], c=colors)#,label=f'$\\langle R^x_G \\rangle_k$')
caps = 5
capt = 2
#for x, y, err, color in zip(xs, list(N8N32_exclVol_moments[1]), list(N8N32_exclVol_moments_dev[0]), colors):
    #ax_rg[0].errorbar(x, y, yerr=err, c=color, capsize=caps, capthick=capt)
ax_rg[1].scatter(xs, N8N32_exclVol_moments[3], c=colors)
#for x, y, err, color in zip(xs, list(N8N32_exclVol_moments[3]), list(N8N32_exclVol_moments_dev[1]), colors):
    #ax_rg[1].errorbar(x, y, yerr=err, c=color, capsize=caps, capthick=capt)
ax_rg[0].set_title('N=8')
ax_rg[1].set_title('N=32')
xmin = 1
for ind, m in enumerate(N8N32_exclVol_moments[0]):
    ax_rg[0].hlines(xmin=xmin, xmax=xmin+2, y=m, ls='--', color=colors[ind], label=f'$\\langle R^{2*ind+2}_G \\rangle_0$')
    xmin = 2*ind + 3
xmin = 1    
for ind, m in enumerate(N8N32_exclVol_moments[3]):
    ax_rg[1].hlines(xmin=xmin, xmax=xmin+2, y=m, ls='--', color=colors[ind], label=f'$\\langle R^{2*ind+2}_G \\rangle_0$')
    xmin = 2*ind + 3
    
ax_rg[0].set_xticks([2, 4, 6])
ax_rg[0].set_xlabel('x')
ax_rg[0].set_ylabel('$\\langle R_G^x \\rangle$')
ax_rg[1].set_xticks([2, 4, 6])
ax_rg[1].set_xlabel('x')
ax_rg[1].set_ylabel('$\\langle R_G^x \\rangle$')
ax_rg[0].legend(loc='upper left', bbox_to_anchor=(0,1))
fig_rg.set_size_inches(10, 5)
fig_rg.tight_layout()

# bitwise dist plot
fig_bwiseDist, ax_bwiseDist = plt.subplots(1, 2)
fig_bwiseDist.set_size_inches(10, 5)

ax_bwiseDist[0].plot(np.linspace(1, len(lists[0][-2]), int(len(lists[0][-2]))), 
                     interpolate_gaps(lists[0][-2])/(2*(8-2+1)))
ax_bwiseDist[0].plot(np.linspace(1, len(bwise_dist_small), int(len(bwise_dist_small))), 
                     interpolate_gaps(bwise_dist_small)/(2*(8-2+1)))
ax_bwiseDist[0].scatter(np.linspace(1, len(lists[0][-2]), int(len(lists[0][-2]))), lists[0][-2]/(2*(8-2+1)), s=10, label='$n_{hidden}=100$')
ax_bwiseDist[0].scatter(np.linspace(1, len(bwise_dist_small), int(len(bwise_dist_small))), bwise_dist_small/(2*(8-2+1)), s=10, label='$n_{hidden}=7$')
ax_bwiseDist[0].set_title('N=8')
ax_bwiseDist[0].set_xlabel('k')
ax_bwiseDist[0].set_ylabel('$\\langle \\beta_k \\rangle$')
ax_bwiseDist[0].axhline(y=bitwise_dist_twoSets(lists[0][0]), c='red', ls='dotted', label='$\\langle \\beta_k \\rangle_{data}$')

ax_bwiseDist[1].plot(np.linspace(1, len(lists[1][-2]), int(len(lists[1][-2]))), 
                     interpolate_gaps(lists[1][-2])/(2*(32-2+1)))
ax_bwiseDist[1].scatter(np.linspace(1, len(lists[1][-2]), int(len(lists[1][-2]))), lists[1][-2]/(2*(32-2+1)), s=10)
ax_bwiseDist[1].set_title('N=32')
ax_bwiseDist[1].set_xlabel('k')
ax_bwiseDist[1].set_ylabel('$\\langle \\beta_k \\rangle$')
ax_bwiseDist[1].axhline(y=bitwise_dist_twoSets(lists[1][0]), c='red', ls='dotted', label='$\\langle \\beta_k \\rangle_{data}$')

ax_bwiseDist[1].set_ylim(0, 0.6)
ax_bwiseDist[0].set_ylim(0, 0.6)
print(f'overlap fraction for N=8: {lists[0][-1]}\
    \n overlap fraction for N=32: {lists[1][-1]}')

ax_bwiseDist[0].set_xticks([1, 200, 400, 600, 800])
ax_bwiseDist[1].set_xticks([1, 200, 400, 600, 800])
ax_bwiseDist[0].legend(loc='upper right')
ax_bwiseDist[1].legend(loc='lower right')

fig_bwiseDist.tight_layout()

### saving of figures

In [ ]:
fig_path = ###path to save the figures for all plots###

fig_corr.savefig(fig_path+'/N8N32_rings_corr.pdf', format='pdf', dpi=1200)
fig_loops.savefig(fig_path+'/N8N32_rings_loops.pdf', format='pdf', dpi=1200)
fig_rg.savefig(fig_path+'/N8N32_rings_gammas.pdf', format='pdf', dpi=1200)
fig_bwiseDist.savefig(fig_path+'/N8N32_rings_dist.pdf', format='pdf', dpi=1200)

### sample loop counts and end-to-end distances for all network sizes

In [ ]:
# load loop counts
savepath = savepath_8
savepath_big = savepath_32

loops_8 = []
ete_8 = []
loops_32 = []
ete_32 = []
networkSizes = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 300]
networkSizes_big = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 300, 400, 500]

for n in networkSizes:
    run8 = f'rw_paper_exclVol_N8_h{n}'
    rfl, ete = sample_loops(savepath, run8, 8+1, ete=True)
    loops_8.append(rfl)
    ete_8.append(ete)

for N in networkSizes_big:
    run32 = f'rw_exvol_ring_N32_h{N}'
    rfl, ete = sample_loops(savepath_big, run32, 32+1, ete=True)
    loops_32.append(rfl)
    ete_32.append(ete)

### plot loop counts and end-to-end distance as function of network size

In [ ]:
markers = ['.', 'o', 'v', '8', 'P', '*']
plt.rcParams.update({'font.size': 18})
nws = [networkSizes, networkSizes_big]

not_zero_8 = np.logical_not((np.array(loops_8)==0).all(axis=0))
not_zero_32 = np.logical_not((np.array(loops_32)==0).all(axis=0))
loop8 = np.array(loops_8)[:, not_zero_8]
loop32 = np.append((np.array(loops_32)[:, not_zero_32])[:, :4], (np.array(loops_32)[:, not_zero_32])[:, -1:], axis=1)
loops = [loop8, loop32]
etes = [ete_8, ete_32]
not_zero = [not_zero_8, not_zero_32]

fig_ete, axs_ete = plt.subplots(1, 2)
fig_ete.set_size_inches(10, 5)

for ind, N in enumerate([8, 32]):
    for i in range(len(loops[ind][0, :])):
        if N==32 and i+1==len(loops[ind][0, :]):
            ax[ind].scatter(nws[ind], loops[ind][:, i], 
                        label='l=32')
            print(loops[ind][:, i])
        else:
            ax[ind].scatter(nws[ind], loops[ind][:, i], 
                        label=f'l={np.where(not_zero[ind]==True)[0][i]}')
        ax[ind].plot(nws[ind], loops[ind][:, i])
        ax[ind].set_title(f'N={N}')
        ax[ind].set_xscale('log')
        ax[ind].legend()
    axs_ete[ind].scatter(nws[ind], etes[ind])
    axs_ete[ind].plot(nws[ind], etes[ind])
    axs_ete[ind].set_title(f'N={N}')
    axs_ete[ind].set_xlabel('$n_{hidden}$')
    axs_ete[ind].set_ylabel('$\\langle R_E \\rangle$')
    axs_ete[ind].set_xscale('log')

ax[1].axvline(x=100, ls='dotted', c='red')
axs_ete[1].axvline(x=100, ls='dotted', c='red')
axs_ete[0].set_yticks([0, 1, 2, 3])
axs_ete[1].set_yticks([0, 5, 10])
ax[0].set_xlabel('$n_{hidden}$')
ax[0].set_ylabel('loops per chain')
ax[1].set_xlabel('$n_{hidden}$')
ax[1].set_ylabel('loops per chain')

fig_ete.tight_layout()

fig.set_size_inches(10, 5)
fig.tight_layout()

"""fig.savefig('bachelorarbeit_maximilian_schwabe/plot_svg/N8N32_rings_nwconvergence.svg', format='svg', dpi=1200)
fig.savefig('bachelorarbeit_maximilian_schwabe/plot_svg/N8N32_rings_nwconvergence.pdf', format='pdf', dpi=1200)"""

fig.savefig('code_BA_Arbeit/plot_svg/N8N32_rings_nwconvergence.svg', format='svg', dpi=1200)
fig.savefig('code_BA_Arbeit/plot_svg/N8N32_rings_nwconvergence.pdf', format='pdf', dpi=1200)

fig_ete.savefig('code_BA_Arbeit/plot_svg/N8N32_rings_endtoend.pdf', format='pdf', dpi=1200)

## evaluation of linear chains

> although many variables still bear the name "exclVol...", all functions work identically, if one uses simulation data and training results of e.g. non-returning or ideal chains


### loading of reconstructions & calculation of observables

In [ ]:
savepath = ###directory of training results, assuming it to be shared for all chain lengths###

savepath_8 = ###directory of training results for N=8###
savepath_32 = ###directory of training results for N=32###

run8 = ###unique identifier of training run for N=8 chain###
run32 = ###unqiue identifier of training run for N=32 chain###

fpath_8 = ###file location of training data for N=8###
fpath_32 = ###file location of training data for N=32###

In [ ]:
Ns = [8, 32]
list8 = []
list32 = []
lists = [list8, list32]
runs = [run8, run32]
fnames = [fpath_8, fpath_32]
refloops = []

for enum, N in enumerate(Ns):
    # create sample data
    rbm = anaU.loader(savepath, runs[enum], rbm=True)
    all_samples = []
    for param in rbm.parameters():
        param.requires_grad = False
    
    print(f'start sampling from {runs[enum]} RBM')
    for i in tqdm(range(100)):
        initialize = torch.from_numpy(np.random.binomial(1, 0.5, (100, (N-2)*2))).float()
        rbm.k = 800
        samples = rbm(initialize)[2].detach().numpy()
        all_samples.append(samples)
    
    samples = np.concatenate(all_samples)
    samplesTF = polyRBM.makeVecs(polyRBM.bitsToBonds2DYu(samples))

    # load raw data
    fname = fnames[enum]
    dataset = polyRBM.CubicChainDataset(fname, 2)

    ind = np.linspace(0, len(dataset)-1, len(dataset)).astype(int)
    np.random.shuffle(ind)

    dataset.data = dataset.data[ind]
    dataset.bits = dataset.bits[ind]
    dataset.bonds = dataset.bonds[ind]

    total = int(1*len(dataset)) # adjust size of data set used for calculating reference values for plots
    div = int(0.7*total)
    testset = dataset.bits[div:total]
    dataset.data = dataset.data[:div]
    dataset.bonds = dataset.bonds[:div]
    dataset.bits = dataset.bits[:div]
    
    # calculate observables
    bwise_dist = bitwise_dist(dataset.data, rbm, 1)
    overlap = not_in_set(dataset.data.detach().numpy(), samples)
    
    data = polyRBM.makeVecs(dataset.bonds)
    
    refloops.append(anaU.loop_count(dataset.bits)[anaU.loop_count(dataset.bits)!=0])
    lists[enum].append(dataset.bits)
    lists[enum].append(data)
    lists[enum].append(samples)
    lists[enum].append(samplesTF)  
    lists[enum].append(bwise_dist)
    lists[enum].append(overlap)

### calculation of observables and plot

In [ ]:
# calculate bondvector correlation and gammas (and save as 2d-array)
plt.rcParams.update({'font.size': 18})
N8N32_exclVol_corr = []
N8N32_exclVol_moments = []

for l in lists:
    N8N32_exclVol_corr.append(anaU.bondvec_correlation(l[1], len(l[1]))) #data
    N8N32_exclVol_corr.append(anaU.bondvec_correlation(l[3], len(l[3]))) #samples
    
    rg2_samples = anaU.rgx_calc(l[3], len(l[3]))
    rg2_data = anaU.rgx_calc(l[1], len(l[1]))

    tp1S = np.mean(rg2_samples[-1])
    tp1D = np.mean(rg2_data[-1])
    tp2S = np.mean(rg2_samples[-1]**2)
    tp2D = np.mean(rg2_data[-1]**2)
    tp3S = np.mean(rg2_samples[-1]**3)
    tp3D = np.mean(rg2_data[-1]**3)
   
    momentsS = [tp1S, tp2S, tp3S]
    momentsD = [tp1D, tp2D, tp3D]
    N8N32_exclVol_moments.append(momentsD)
    N8N32_exclVol_moments.append(momentsS)
    
# load and plot training loop figures
loops_8 = anaU.loader(savepath, run8, loops=True)
loops_32 = anaU.loader(savepath, run32, loops=True)

not_zero_8 = np.logical_not((np.array(loops_8)==0).all(axis=0))
not_zero_32 = np.logical_not((np.array(loops_32)==0).all(axis=0))
loop8 = np.array(loops_8)[:, not_zero_8]
loop32 = (np.array(loops_32)[:, not_zero_32])[:, :4]
fig, axs = plt.subplots(2)
axs[0].imshow(loop8[:10])
axs[1].imshow(loop32[:10])

fig_loops, ax_loops = plt.subplots(1, 2)
rbm_epochs = len(loop8)

for i in range(len(loop8[0, :])):
    loop8[:, i][loop8[:, i]==0] = None
    ax_loops[0].plot(np.linspace(0, rbm_epochs, rbm_epochs), loop8[:, i], label=f'l={np.where(not_zero_8==True)[0][i]}')

for i in range(len(loop32[0, :])):
    loop32[:, i][loop32[:, i]==0] = None
    ax_loops[1].plot(np.linspace(0, rbm_epochs, rbm_epochs), loop32[:, i], label=f'l={np.where(not_zero_32==True)[0][i]}')
    
ax_loops[0].set_title('N=8')
ax_loops[1].set_title('N=32')
ax_loops[0].set_xlim(-2, rbm_epochs)
ax_loops[1].set_xlim(-2, rbm_epochs)
ax_loops[0].set_xlabel('epochs')
ax_loops[0].set_ylabel('loops per chain')
ax_loops[1].set_xlabel('epochs')
ax_loops[1].set_ylabel('loops per chain')
ax_loops[0].legend(loc='upper right', bbox_to_anchor=(1,0.8))
ax_loops[1].legend(loc='upper right', bbox_to_anchor=(1,0.9))
fig_loops.set_size_inches(10, 5)
fig_loops.tight_layout()

# plot bondvec corr
fig_corr, ax_corr = plt.subplots(1, 2)
ax_corr[0].plot(np.linspace(0, len(N8N32_exclVol_corr[0])-1, len(N8N32_exclVol_corr[0])), N8N32_exclVol_corr[0])
ax_corr[0].scatter(np.linspace(0, len(N8N32_exclVol_corr[0])-1, len(N8N32_exclVol_corr[0])), N8N32_exclVol_corr[0],
                  label='data')
ax_corr[0].plot(np.linspace(0, len(N8N32_exclVol_corr[1])-1, len(N8N32_exclVol_corr[1])), N8N32_exclVol_corr[1])
ax_corr[0].scatter(np.linspace(0, len(N8N32_exclVol_corr[1])-1, len(N8N32_exclVol_corr[1])), N8N32_exclVol_corr[1],
                  label='reconstruction')
ax_corr[0].set_title('N=8')
ax_corr[0].set_xlabel('|i-j|')
ax_corr[0].set_ylabel('$\\left\\langle\\frac{\\vec{b}_i \cdot \\vec{b}_j}{|\\vec{b}_i|\cdot|\\vec{b}_j|}\\right\\rangle$')
ax_corr[0].legend()

ax_corr[1].plot(np.linspace(0, len(N8N32_exclVol_corr[2])-1, len(N8N32_exclVol_corr[2])), N8N32_exclVol_corr[2])
ax_corr[1].scatter(np.linspace(0, len(N8N32_exclVol_corr[2])-1, len(N8N32_exclVol_corr[2])), N8N32_exclVol_corr[2])
ax_corr[1].plot(np.linspace(0, len(N8N32_exclVol_corr[-1])-1, len(N8N32_exclVol_corr[-1])), N8N32_exclVol_corr[3])
ax_corr[1].scatter(np.linspace(0, len(N8N32_exclVol_corr[-1])-1, len(N8N32_exclVol_corr[-1])), N8N32_exclVol_corr[3])
ax_corr[1].set_title('N=32')
ax_corr[1].set_xlabel('|i-j|')
ax_corr[0].set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
ax_corr[1].set_xticks([0, 5, 10, 15, 20, 25, 30])

fig_corr.set_size_inches(10, 5)
fig_corr.tight_layout()


# plot moments of Rg^2
xs = [2, 4, 6]
colors = ['red', 'orange', 'green']
fig_rg, ax_rg = plt.subplots(1, 2)
ax_rg[0].scatter(xs, N8N32_exclVol_moments[1], c=colors)
ax_rg[1].scatter(xs, N8N32_exclVol_moments[3], c=colors)
ax_rg[0].set_title('N=8')
ax_rg[1].set_title('N=32')
xmin = 1
for ind, m in enumerate(N8N32_exclVol_moments[0]):
    ax_rg[0].hlines(xmin=xmin, xmax=xmin+2, y=m, ls='--', color=colors[ind], label=f'$\\langle R^{2*ind+2}_G \\rangle_0$')
    xmin = 2*ind + 3
xmin = 1    
for ind, m in enumerate(N8N32_exclVol_moments[3]):
    ax_rg[1].hlines(xmin=xmin, xmax=xmin+2, y=m, ls='--', color=colors[ind], label=f'$\\langle R^{2*ind+2}_G \\rangle_0$')
    xmin = 2*ind + 3
    
ax_rg[0].set_xticks([2, 4, 6])
ax_rg[0].set_xlabel('x')
ax_rg[0].set_ylabel('$\\langle R_G^x \\rangle$')
ax_rg[1].set_xticks([2, 4, 6])
ax_rg[1].set_xlabel('x')
ax_rg[1].set_ylabel('$\\langle R_G^x \\rangle$')
ax_rg[0].legend(loc='upper left', bbox_to_anchor=(0,1))
fig_rg.set_size_inches(10, 5)
fig_rg.tight_layout()

# plot bitwise distance
fig_bwiseDist, ax_bwiseDist = plt.subplots(1, 2)
fig_bwiseDist.set_size_inches(10, 5)

ax_bwiseDist[0].plot(np.linspace(1, len(lists[0][-2]), int(len(lists[0][-2]))), 
                     interpolate_gaps(lists[0][-2])/(2*(8-2)))
ax_bwiseDist[0].scatter(np.linspace(1, len(lists[0][-2]), int(len(lists[0][-2]))), lists[0][-2]/(2*(8-2)), s=10)
ax_bwiseDist[0].set_title('N=8')
ax_bwiseDist[0].set_xlabel('k')
ax_bwiseDist[0].set_ylabel('$\\langle \\beta_k \\rangle$')
ax_bwiseDist[0].axhline(y=bitwise_dist_twoSets(lists[0][0]), c='red', ls='dotted', label='$\\langle \\beta_k \\rangle_{data}$')

ax_bwiseDist[1].plot(np.linspace(1, len(lists[1][-2]), int(len(lists[1][-2]))), 
                     interpolate_gaps(lists[1][-2])/(2*(32-2)))
ax_bwiseDist[1].scatter(np.linspace(1, len(lists[1][-2]), int(len(lists[1][-2]))), lists[1][-2]/(2*(32-2)), s=10)
ax_bwiseDist[1].set_title('N=32')
ax_bwiseDist[1].set_xlabel('k')
ax_bwiseDist[1].set_ylabel('$\\langle \\beta_k \\rangle$')
ax_bwiseDist[1].axhline(y=bitwise_dist_twoSets(lists[1][0]), c='red', ls='dotted', label='$\\langle \\beta_k \\rangle_{data}$')
ax_bwiseDist[1].set_ylim(0, 0.6)
ax_bwiseDist[0].set_ylim(0, 0.6)
print(f'overlap fraction for N=8: {lists[0][-1]}\
    \n overlap fraction for N=32: {lists[1][-1]}')

ax_bwiseDist[0].set_xticks([1, 200, 400, 600, 800])
ax_bwiseDist[1].set_xticks([1, 200, 400, 600, 800])
ax_bwiseDist[0].legend(loc='lower right')
ax_bwiseDist[1].legend(loc='lower right')

fig_bwiseDist.tight_layout()

### calculate linear fit of logarithmic bondvec corr. and plot it

In [ ]:
fig_loglog_corr, ax_loglog_corr = plt.subplots(1, 2)
N8_corr_dat = N8N32_exclVol_corr[0]
N8_corr_s = N8N32_exclVol_corr[1]
N32_corr_dat = N8N32_exclVol_corr[2]
N32_corr_s = N8N32_exclVol_corr[3]
N32_corr_dat[N32_corr_dat<10**-10] = None
N32_corr_s[N32_corr_s<10**-10] = None

fits = []

ax_loglog_corr[0].scatter(np.linspace(0.0001, len(N8_corr_dat)-1, len(N8_corr_dat)),
                          np.log(N8_corr_dat), label='data')
fit_N8_d = stats.linregress(np.linspace(0.0001, len(N8_corr_dat)-1, len(N8_corr_dat)), np.log(N8_corr_dat))
ax_loglog_corr[0].plot(np.linspace(0.0001, len(N8_corr_dat)-1, len(N8_corr_dat)),
                      np.linspace(0.0001, len(N8_corr_dat)-1, len(N8_corr_dat))*fit_N8_d.slope + fit_N8_d.intercept,
                      linewidth=5)

ax_loglog_corr[0].scatter(np.linspace(0.0001, len(N8_corr_s)-1, len(N8_corr_s)), 
                          np.log(N8_corr_s), label='reconstruction')
fit_N8_s = stats.linregress(np.linspace(0.0001, len(N8_corr_s)-1, len(N8_corr_s)), np.log(N8_corr_s))
ax_loglog_corr[0].plot(np.linspace(0.0001, len(N8_corr_dat)-1, len(N8_corr_dat)),
                      np.linspace(0.0001, len(N8_corr_dat)-1, len(N8_corr_dat))*fit_N8_d.slope + fit_N8_d.intercept)

N32_corr_dat = abs(N32_corr_dat[~np.isnan(N32_corr_dat)])
ax_loglog_corr[1].scatter(np.linspace(0.0001, len(N32_corr_dat)-1, len(N32_corr_dat)), 
                          np.log(N32_corr_dat))
fit_N32_d = stats.linregress(np.linspace(0.0001, len(N32_corr_dat)-1, len(N32_corr_dat)), np.log(N32_corr_dat))
ax_loglog_corr[1].plot(np.linspace(0.0001, len(N32_corr_dat)-1, len(N32_corr_dat)),
                      np.linspace(0.0001, len(N32_corr_dat)-1, len(N32_corr_dat))*fit_N32_d.slope 
                       + fit_N32_d.intercept, label='$\\propto c_{data}$')

N32_corr_s = N32_corr_s[~np.isnan(N32_corr_s)]
ax_loglog_corr[1].scatter(np.linspace(0.0001, len(N32_corr_s)-1, len(N32_corr_s)), 
                          np.log(N32_corr_s))
fit_N32_s = stats.linregress(np.linspace(0.0001, len(N32_corr_s)-1, len(N32_corr_s)), np.log(N32_corr_s))
ax_loglog_corr[1].plot(np.linspace(0.0001, len(N32_corr_s)-1, len(N32_corr_s)),
                      np.linspace(0.0001, len(N32_corr_s)-1, len(N32_corr_s))*fit_N32_s.slope
                       + fit_N32_s.intercept, label='$\\propto c_{recon}$')

ax_loglog_corr[0].set_yticks([0, -1, -2, -3, -4])
ax_loglog_corr[1].set_yticks([0, -2, -4, -6, -8, -10])

ax_loglog_corr[0].set_title('N=8')
ax_loglog_corr[1].set_title('N=32')

ax_loglog_corr[0].set_xlabel('|i-j|')
ax_loglog_corr[0].set_ylabel('$\\log{\\left\\langle\\frac{\\vec{b}_i \cdot \\vec{b}_j}{|\\vec{b}_i|\cdot|\\vec{b}_j|}\\right\\rangle}$')
ax_loglog_corr[1].set_xlabel('|i-j|')

ax_loglog_corr[0].legend(loc='upper right')
ax_loglog_corr[1].legend(loc='upper right')

fig_loglog_corr.set_size_inches(10, 5)
fig_loglog_corr.tight_layout()

print(f'data fit for N=8: c={fit_N8_d.slope}+-{fit_N8_d.stderr}; sample fit for N=8: c={fit_N8_s.slope}+-{fit_N8_s.stderr}\
\n data fit for N=32: c={fit_N32_d.slope}+-{fit_N32_d.stderr}; sample fit for N=32: c={fit_N32_s.slope}+-{fit_N32_s.stderr}')

### saving of all figures

In [ ]:
fig_path = ###path to save all plots###

fig_corr.savefig(fig_path+'/N8N32_exclVol_corr.pdf', format='pdf', dpi=1200)
fig_loglog_corr.savefig(fig_path+'/N8N32_exclVol_logcorr.pdf', format='pdf', dpi=1200)
fig_loops.savefig(fig_path+'/N8N32_exclVol_loops.pdf', format='pdf', dpi=1200)
fig_rg.savefig(fig_path+'/N8N32_exclVol_gammas.pdf', format='pdf', dpi=1200)
fig_bwiseDist.savefig(fig_path+'/N8N32_exclVol_dist.pdf', format='pdf', dpi=1200)

### calculation of loop counts for all network sizes

In [ ]:
# load loop counts
savepath = savepath_8
savepath_big = savepath_32

loops_8 = []
loops_32 = []
networkSizes = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 300]
networkSizes_big = [1, 2, 3, 4, 5, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 100, 200, 300, 400, 500]

for n in networkSizes:
    run8 = f'rw_paper_exclVol_N8_h{n}'
    loops_8.append(sample_loops(savepath, run8, 8))
   
for N in networkSizes_big:
    run32 = f'rw_exclVol_N32_h{N}'
    loops_32.append(sample_loops(savepath_big, run32, 32))

### plot of loop counts as function of network sizes

In [ ]:
markers = ['.', 'o', 'v', '8', 'P', '*']
nws = [networkSizes, networkSizes_big]

fig, ax = plt.subplots(1, 2)
plt.rcParams.update({'font.size': 18})

not_zero_8 = np.logical_not((np.array(loops_8)==0).all(axis=0))
not_zero_32 = np.logical_not((np.array(loops_32)==0).all(axis=0))
loop8 = np.array(loops_8)[:, not_zero_8]
loop32 = (np.array(loops_32)[:, not_zero_32])[:, :4]
loops = [loop8, loop32]
not_zero = [not_zero_8, not_zero_32]

for ind, N in enumerate([8, 32]):
    for i in range(len(loops[ind][0, :])):
        ax[ind].scatter(nws[ind], loops[ind][:, i], 
                        label=f'l={np.where(not_zero[ind]==True)[0][i]}')
        ax[ind].plot(nws[ind], loops[ind][:, i])
        ax[ind].set_title(f'N={N}')
        ax[ind].set_xscale('log')
        ax[ind].legend()

ax[1].set_ylim(0, 3)
ax[0].set_ylim(0, 1)

ax[0].set_xlabel('$n_{hidden}$')
ax[0].set_ylabel('loops per chain')
ax[1].set_xlabel('$n_{hidden}$')
ax[1].set_ylabel('loops per chain')

fig.set_size_inches(10, 5)
fig.tight_layout()

fig.savefig(fig_path+'/N8N32_exclVol_nwconvergence.svg', format='svg', dpi=1200)
fig.savefig(fig_path+'/N8N32_exclVol_nwconvergence.pdf', format='pdf', dpi=1200)